In [ ]:
import subprocess
import sys,os

Instalasi library yang dibutuhkan dengan menggunakan requirements.txt

In [ ]:
pip install --upgrade pip setuptools wheel

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.layers import GlobalAveragePooling2D
import os
import json
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras.applications import VGG16

Path ke direktori dataset

In [ ]:
from google.colab import drive

# Menghubungkan ke Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_dir = '/content/drive/MyDrive/Coconut Tree Disease Dataset'

Membuat ImageDataGenerator untuk memuat gambar dengan augmentasi data tambahan<br>
Jangan dirubah karna ini adalah yang paling bagus,kalau mau di rubah silakan research lagi

In [ ]:
datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Generator untuk training data

In [ ]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 3661 images belonging to 5 classes.


Generator untuk validation data (tidak menggunakan augmentasi data)

In [ ]:
validation_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 913 images belonging to 5 classes.


Membuat model yang lebih kompleks

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
#Model Resnet 50 (punya affandra)
#sil kalo misalnya mau pake callback, syntax nya ada di model di cell paling bawahh

resnet_model = tf.keras.applications.ResNet50(input_shape=(150, 150, 3), include_top = False, weights = 'imagenet')
resnet_model.trainable = False

In [ ]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
model = Sequential([
    resnet_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(1025, activation = 'relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

TypeError: The added layer must be an instance of class Layer. Received: layer=<class 'keras.src.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D'> of type <class 'type'>.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Melatih model

In [ ]:
history = model.fit(
    train_generator,
    epochs=100,  # Meningkatkan jumlah epoch untuk pelatihan yang lebih baik
    validation_data=validation_generator
)

Epoch 1/100
115/115 [==============================] - 513s 4s/step - loss: 2.2252 - accuracy: 0.3478 - val_loss: 1.5075 - val_accuracy: 0.3228
Epoch 2/100
115/115 [==============================] - 511s 4s/step - loss: 1.3285 - accuracy: 0.3959 - val_loss: 1.2706 - val_accuracy: 0.4234
Epoch 3/100
115/115 [==============================] - 512s 4s/step - loss: 1.1530 - accuracy: 0.5040 - val_loss: 1.1211 - val_accuracy: 0.5503
Epoch 4/100
115/115 [==============================] - 499s 4s/step - loss: 0.9704 - accuracy: 0.6074 - val_loss: 0.8811 - val_accuracy: 0.6575
Epoch 5/100
115/115 [==============================] - 509s 4s/step - loss: 0.8773 - accuracy: 0.6631 - val_loss: 0.7198 - val_accuracy: 0.6433
Epoch 6/100
115/115 [==============================] - 510s 4s/step - loss: 0.8026 - accuracy: 0.7158 - val_loss: 0.6574 - val_accuracy: 0.6827
Epoch 7/100
115/115 [==============================] - 506s 4s/step - loss: 0.6737 - accuracy: 0.7576 - val_loss: 1.1173 - val_accuracy:

KeyboardInterrupt: 

Mengevaluasi model

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

Menyimpan model

In [ ]:
model.save('model/image_classification_model.h5')

NameError: name 'model' is not defined

Menyimpan class indices

In [ ]:
class_indices = train_generator.class_indices
with open('model/class_indices.json', 'w') as f:
    json.dump(class_indices, f)

In [ ]:
#MODEL COBA COBA, model fix ada di atas

In [ ]:
from keras.layers import GlobalAveragePooling2D
ResNet_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
ResNet_model.trainable = False


234545216/234545216 [==============================] - 2s 0us/step


In [ ]:
ResNet_model.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 77, 77, 64)           0         ['conv1_conv[0][0]']          
                                                                                        

In [ ]:
model = Sequential([
    ResNet_model,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(1025, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='RMSProp',
              metrics=['accuracy'])

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('val_accuracy') > 0.93):
                print('\nStop training model...')
                self.model.stop_training = True

    # Inisiasi class MyCallback ke dalam variable
custom_callback = MyCallback()
model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
        callbacks=[custom_callback, tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)])

Epoch 1/100
115/115 [==============================] - 1653s 14s/step - loss: 0.5982 - accuracy: 0.8506 - val_loss: 0.7648 - val_accuracy: 0.8105 - lr: 0.0010
Epoch 2/100
115/115 [==============================] - 1149s 10s/step - loss: 0.1827 - accuracy: 0.9405 - val_loss: 0.6173 - val_accuracy: 0.8467 - lr: 0.0010
Epoch 3/100
115/115 [==============================] - 1096s 10s/step - loss: 0.1329 - accuracy: 0.9525 - val_loss: 0.7307 - val_accuracy: 0.8215 - lr: 0.0010
Epoch 4/100
115/115 [==============================] - 1135s 10s/step - loss: 0.1306 - accuracy: 0.9615 - val_loss: 0.6513 - val_accuracy: 0.8488 - lr: 0.0010
Epoch 5/100
115/115 [==============================] - 1138s 10s/step - loss: 0.1012 - accuracy: 0.9675 - val_loss: 0.9066 - val_accuracy: 0.8445 - lr: 0.0010
Epoch 6/100
115/115 [==============================] - 1076s 9s/step - loss: 0.0976 - accuracy: 0.9719 - val_loss: 0.9882 - val_accuracy: 0.8554 - lr: 0.0010
Epoch 7/100
115/115 [==========================

In [ ]:
i

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

NameError: name 'model' is not defined